## StarTracker WCS solutions - 17-Nov-22

Craig Lage - basically copying code from Merlin Fisher-Levine

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.blindSolving import runImchar, genericCameraHeaderToWcs, CommandLineSolver

In [ ]:
solver = CommandLineSolver(indexFiles='/scratch/mfisherlevine/astrometry_net/broiler.astrometry.net/~dstn/4100',
                           checkInParallel=True)

In [ ]:
# Dates are still hard coded.
path = '/project/GenericCamera/101/2022/11/16/'

In [ ]:
seqNums = []
deltaAzs = []
deltaEls = []
outfile = open("/home/craiglagegit/DATA/StarTracker_Summary_Rot_20221116.txt", "w")
outfile.write("SeqNum RA(header) Dec(header) RA(Solve) Dec(Solve) Az(hdr-fix) El(header) Az(solve) El(solve) DeltaAz DeltaEl Rot\n")
for seqNum in range(4, 119+1):
    seqNums.append(seqNum)
    filename = path + f'GC101_O_20221116_{seqNum:06}.fits'
    exp = afwImage.ExposureF(filename)
    wcs = genericCameraHeaderToWcs(exp)
    exp.setWcs(wcs)
    results = runImchar(exp, 5, 25)
    newWcs = solver.run(exp, results.sourceCat, silent=True)
    raSolve = newWcs.getSkyOrigin().getRa().asDegrees()
    decSolve = newWcs.getSkyOrigin().getDec().asDegrees()
    hdulist = pf.open(filename)
    header = hdulist[0].header
    raStart = header["RASTART"]
    decStart = header["DECSTART"]
    azStart = header["AZSTART"]
    azStart = 360.0 - azStart # Compensating for Az sign error
    elStart = header["ELSTART"]
    rotpa = header["ROTPA"]
    rot = newWcs.getRelativeRotationToWcs(wcs).asDegrees()
    time = Time(header["DATE-OBS"])
    altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
    skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
    obsAltAz = skyLocation.transform_to(altAz)
    az = obsAltAz.az.deg
    el = obsAltAz.alt.deg
    deltaAz = azStart - az
    deltaEl = elStart - el
    deltaAzs.append(deltaAz)
    deltaEls.append(deltaEl)
    outfile.write(f"{seqNum}      {raStart:.3f}    {decStart:.3f}      {raSolve:.3f}    {decSolve:.3f}    {azStart:.3f}      {elStart:.3f}    {az:.3f}    {el:.3f}   {deltaAz:.3f}   {deltaEl:.3f}  {rot:.3f}\n")
    print(f"{seqNum}, deltaAz = {deltaAz:.3f}, deltaEl = {deltaEl:.3f}, rotpa = {rotpa}, rot = {newWcs.getRelativeRotationToWcs(wcs)}")
outfile.close()
    